In [ ]:
import numpy as np
import pandas as pd
import os
import tarfile
import urllib

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16,10)

# scikit-learn

Scikit-Learn is one of the most widely used python modules for machine learning. It has implementations of a huge number of algorithms and helper tools and (as far as possible) provides a consistent and easy to use interface to them. 

_**Caveat**: Machine learning is a _huge_ topic and I'm not an expert in it. These notes cover some introductory steps and useful tips, but you should augment them with other resources as you learn more. The [scikit-learn tutorial](https://scikit-learn.org/stable/tutorial/index.html) is a good place to start, and there are lots of [courses](https://medium.com/javarevisited/5-best-scikit-learn-online-courses-for-machine-learning-and-data-science-6beb02e9cca0) and [books](https://www.amazon.ca/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1492032646/ref=sr_1_1?keywords=hands-on+machine+learning+with+scikit-learn%2C+keras%2C+and+tensorflow&qid=1657479285&sprefix=scikit-learn%2Caps%2C425&sr=8-1) available._

## The Problem

Machine Learning problems usually start with a (potentially large) collection of data which you want to use to draw inferences from to make predictions. There are two broad categories of machine learning algorithms:

  * **Supervised Learning**: In supervised learning, the data you are learning from comes with the answers: it is _labeled_. The idea is to use this labeling to train your algorithm to make predictions for new data which lack the labels. There are two broad categories of problem where supervised learning is used:
  
      * **Classification Problems**: where you want to use the data to divide instances into discrete categories (e.g. Logistic Regression)
      * **Regression Problems**: where you want to use the data to make some numerical predictions (e.g. Linear Regression)
      
      
  * **Unsupervised Learning**: In unsupervised learning the learning algorithm has to learn for itself. It has no labels to guide it, only the data. Some of the most common categories or problem tackled with unsupervised learning algorithms are:
      * **Clustering**: Recognizing distinct groups within the data (e.g. K-Means)
      * **Dimensionality Reduction** Making datasets more tractable while retaining correlations and patterns (e.g. Principal Component Analysis (PCA)
      * **Anomoly/Novelty Detection** Finding outliers in your dataset (e.g. One class Support Vector Machine)

The landscape of machine learning algorithms is huge and continues to grow, we don't have any hope of covering all of them, but if you can categorize your problem similar to those above, some searching and comparing to existing problems will normally help you narrow down the approaches you want to try. Fortunately scikit-learn provides a consistent interface to many of them and makes it possible to explore and experiment until you find the one(s) you need! The Scikit-Learn documentation has a nice [Choosing the right estimator](https://scikit-learn.org/stable/machine_learning_map.html) page.


## A (very basic) scikit-learn recipe

1. Ingest the data
1. Split the data into training and test sets
1. Prepare the data, tidy missing values, scale etc.
1. Train your model
1. Make predictions


We'll work through an example from Aurélien Géron's book [Hands on Machine Learning with scikit-learn](https://www.amazon.ca/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1492032646/ref=sr_1_1?keywords=hands-on+machine+learning+with+scikit-learn%2C+keras%2C+and+tensorflow&qid=1657479285&sprefix=scikit-learn%2Caps%2C425&sr=8-1). **The content below is a repetition of Aurélien's analysis (which is also available as a [notebook](https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb)) with a bit of commentary.**

Starting at step 1, you're experience with `numpy` and `pandas` should help you a lot. We'll assume that the problem set is small enough that it fits in memory on a single machine (e.g. a reasonably small csv) so you can read it in with `pd.read_csv`. For some problems this won't be true and you need to look at algorithms which can work with new data "on-the-fly", these are typically referred to as _online learning_.

The dataset we will use looks at house prices in California and the goal is to try to predict the prices of houses based on other criteria such as the age of the house, the income of the occupents, the number of rooms, etc. In the terminology above, this is a supervised learning regression problem. The next thing to do would be to take a quick look at the data

## 1. Ingest the data

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path, filter='tar')
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
housesDF = pd.read_csv(os.path.join(HOUSING_PATH, "housing.csv"))
housesDF.shape

In [ ]:
housesDF.columns

In [ ]:
housesDF.describe()

In [ ]:
housesDF.head()

In [ ]:
housesDF.dtypes

`ocean_proximity` is a categorical variable...

In [ ]:
housesDF['ocean_proximity'].value_counts()

For the numerical values histograms can be useful to orient yourself with the data

In [ ]:
housesDF.hist(bins=50, figsize=(20, 15))
plt.show()

There are a few things to notice here

1. The values follow various distribution(s)
2. They cover a variety of scales
3. Some values (e.g. `median_house_value` and `housing_median_age` have extreme values which probably represent some sort of cut off or binning

## Split the data into training and test sets

It is important to do this early. The risk is that you'll bias your models and end up with bad generalization errors otherwise. Given that we're going to be playing with feature engineering, scaling and other transformations, this might seem premature, but we'll use Pipelines to address that. At this point, it would also be good to start thinking about selecting a performance measure. For a regression like this RMSE is simple and will probably be sufficient, but the documentation has a section on [metrics and scoring](https://scikit-learn.org/stable/modules/model_evaluation.html) which can help you decide more complicated cases.

When splitting the training and test set, we want to make sure that both sets are sampled without bias. In our example, we are to assume that `median_income` will be important to our final housing price predictions and we want to make sure that our training set includes data from the full range of incomes. Since `median_income` is an (almost) continuous variable, the idea is to bin the values into categories, then use those categories to sample from.

In [ ]:
housesDF['income_cat'] = pd.cut(housesDF['median_income'],
       bins = [0, 1.5, 3.0, 4.5, 6.0, np.inf],
       labels = [1, 2, 3, 4, 5]
)
housesDF['income_cat'].hist()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(housesDF, housesDF['income_cat']):
    trainDForig = housesDF.loc[train_idx]
    testDF = housesDF.loc[test_idx]

trainDF = trainDForig.copy()

In [ ]:
trainDF['income_cat'].hist()

So our training set looks like it samples the various income categories well. We no longer need the `income_cat` column, so we'll drop it from both sets

## 3. Prepare the data, tidy missing values etc.

The histograms for latitude and longitude are probably hiding some structure, all of the values should fall somewhere on a map of California...

In [ ]:
trainDF

In [ ]:
trainDF.plot(
    kind='scatter',
    x='longitude',
    y='latitude',
    alpha=0.4,
    s=trainDF['population']/100,
    label='population',
    c='median_house_value',
    cmap=plt.get_cmap("jet"),
    colorbar=True
)

After taking a look at the individual features, the next thing is to look at combinations of features. Often this can help you prepare the dataset so that the algorithms you select have the best chance for success

In [ ]:
trainDF.corr(numeric_only=True)

In [ ]:
fig, ax = plt.subplots()

ax.matshow(trainDF.corr(numeric_only=True))

To dig a little deeper, you can look at the `scatter_matrix` which plots the numerical values of features against each other 

In [ ]:
from pandas.plotting import scatter_matrix

attributes = [
    'median_house_value',
    'median_income',
    'total_rooms',
    'housing_median_age'
]
scatter_matrix(trainDF[attributes])

The `median_income` shows strong positive correlation with the `median_house_value` (see the [wikipedia entry for correlation](https://en.wikipedia.org/wiki/Correlation)). You can see horizontal lines showing a hard cap of \\$500,000 as well as fainter lines clustered around some other figures (\\$450,000, \\$350,000) etc. These are likely to be artifacts of the way the data was collected or encoded and you should decide what to do with them. For example, it might be worth trying to train your model without the entries valued at \\$500,000 (assuming that the actual values were larger but were encoded as \\$500,000 during data entry).

In [ ]:
trainDF.drop(trainDF[trainDF['median_house_value']>=500000].index, inplace=True)
trainDF.reset_index(inplace=True, drop=True)

In [ ]:
fix, ax = plt.subplots()
ax.scatter(trainDF['median_income'], trainDF['median_house_value'])

It is also useful to look at combinations at attributes. For example, we have the total number of households per district and the total number of bedrooms, but we could combine those to give the number of rooms per household...

In [ ]:
trainDF['rooms_per_household'] = trainDF['total_rooms'] / trainDF['households']
trainDF['bedrooms_per_room'] = trainDF['total_bedrooms'] / trainDF['total_rooms']
trainDF['population_per_household'] = trainDF['population'] / trainDF['households']

In [ ]:
trainDF.corr(numeric_only=True)

To deal with missing values, you can either discard the corresponding data, or replace them with some useful value. Personally, I usually try discarding them first, but if that isn't feasible (throwing away too much valuable data), then scikit-learn has some convenience classes which will help fill in those values.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
median_imputer = SimpleImputer(strategy='median')

trainDF_num = trainDF.drop('ocean_proximity', axis=1)

median_imputer.fit(trainDF_num)

In [ ]:
X = median_imputer.transform(trainDF_num)
type(X)

In [ ]:
trfDF_num = pd.DataFrame(
    X,
    columns=trainDF_num.columns, 
    index=trainDF_num.index
)
trfDF_num.head()

For the categorical values, we will use one hot encoding. You can do this explicitly with `pd.get_dummies`, but similar to what we did with the Imputer, using the corresponding sklearn class lets us reuse the transformation conveniently

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

trainDF_cat = trainDF[['ocean_proximity']]
X = cat_encoder.fit_transform(trainDF_cat)
type(X)

In [ ]:
trfDF_cat = pd.DataFrame(
    X.toarray(),
    columns = cat_encoder.categories_[0],
    index = trainDF_cat.index
)

#### Feature Scaling

Generally, machine learning algorithms want to work with numerical values which share a similar scale. Taking logs etc. can help, but again sklearn also includes some convenience classes, we'll use `StandardScaler` which subtracts the mean and scales to unit variance. At this point, we'll also start building a transformation [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Pipelines are important because your data tidyup and feature engineering is likely to get quite complicated, and your going to need to apply it many, many times while selecting and tuning your models. Packing the transformations up as a pipeline lets us do this reliably 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

X = num_pipeline.fit_transform(trainDF_num)

In [ ]:
trfDF_num = pd.DataFrame(
    X,
    columns=trainDF_num.columns, 
    index=trainDF_num.index
)

In [ ]:
trfDF_num.head()

Now we can build a final pipeline

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

trainDF = trainDForig.copy()
trainDF.drop(trainDF[trainDF['median_house_value']>=500000].index, inplace=True)
trainDF.reset_index(inplace=True, drop=True)

housing_labels = trainDF['median_house_value']
trainDF = trainDF.drop('median_house_value', axis='columns')

rooms_ix      = housesDF.columns.get_loc('total_rooms')
bedrooms_ix   = housesDF.columns.get_loc('total_bedrooms')
population_ix = housesDF.columns.get_loc('population')
households_ix = housesDF.columns.get_loc('households')


class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attr_addr',CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, trainDF.select_dtypes('number').columns),
    ('cat', OneHotEncoder(), trainDF.select_dtypes('object').columns),
])

In [ ]:
trainDF = trainDForig.copy()
trainDF.drop(trainDF[trainDF['median_house_value']>=500000].index, inplace=True)
trainDF.reset_index(inplace=True, drop=True)

housing_labels = trainDF['median_house_value']
trainDF = trainDF.drop('median_house_value', axis='columns')

trf = full_pipeline.fit_transform(trainDF)

tDF = pd.DataFrame(
    full_pipeline.fit_transform(trainDF),
    columns = (
        list(trainDF.select_dtypes('number').columns) + 
        [
            'rooms_per_household',
            'population_per_household',
            'bedrooms_per_room'
        ] + 
        list(full_pipeline['cat'].get_feature_names_out())
    ),
    index = trainDF.index
)

In [ ]:
tDF.head()

## Training a model

After all of the hard work of preparing the data, the consistent design of scikit-learn lets us actually train a model very quickly...

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
housing_prepared = trf.copy()

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = lin_reg.predict(housing_prepared)
housing_predictions

In [ ]:
housing_labels

In [ ]:
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Not great, not terrible. This is saying our predictions are off by around ~\\$60000 for this model. Ideally though, we want to get an idea of how reliable that figure is before proceeding. One way to do this is to hold out some more of our training data for this evaluation (remember we don't want to touch our actual test data until the very end). We could do this a bunch of times, using different segments of the training data and the look at how the root mean square error varies. That's the basic idea behind sklearn's K-fold cross validation feature. We will do a 10 fold cross validation for our model which will split the training data into 10 folds. The model will be trained 10 times in total using 9 of these folds as the training data and the remaining one to evaluate the model

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                         scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-scores)

In [ ]:
print(f"{lin_rmse_scores.mean()} += {lin_rmse_scores.std()}")

At this point we could go back and try to improve the model by adding new features etc. but before we do that we should evaluate some other models to see how they do. Following [Aurélien's analysis](https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb), we'll try a RandomForestRegressor model, notice how similar the steps are for the model, this consistent interface is the main benefit of scikit-learn and rewards all of the work put into the data preparation.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring='neg_mean_squared_error', cv=3)
forest_rmse_scores = np.sqrt(-scores)

In [ ]:
print(f"{forest_rmse_scores.mean()} += {forest_rmse_scores.std()}")

## Hyperparameters



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_.get_params()

We can look at the individual cross validation scores

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

We have some surprisingly good results with e.g. 4 features and 10 estimators, but our best result is with `max_features=8, n_estimators=30`.

One final thing we can do before we use our model is to look at how the various features fed into these results



In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_

extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
#attributes = num_attribs + extra_attribs + cat_one_hot_attribs


sorted(zip(feature_importances, tDF.columns), reverse=True)

Now we can try out our model on the test data

In [ ]:
final_model = grid_search.best_estimator_

X_test = testDF.drop("median_house_value", axis=1)
y_test = testDF["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print(f"{final_rmse}")